Import packages

In [ ]:
import numpy as np
import pandas as pd 
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os

Create constant values

In [ ]:
FAST_RUN = False
IMAGE_WIDTH=192
IMAGE_HEIGHT=256
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3

Add labels to each category

In [ ]:
filenames = os.listdir("./train_data")
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    categories.append(category)

df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

see df

In [ ]:
df

See total in count

In [ ]:
df['category'].value_counts().plot.bar()

Build model with Xception

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization

# Xception用
from keras.applications.xception import Xception
from keras.layers.pooling import GlobalAveragePooling2D

xception=Xception(input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS),include_top=False,weights='imagenet')

x = xception.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation = 'relu')(x)
predictions = Dense(7, activation = 'softmax')(x)

model = Model(inputs = xception.input, outputs = predictions)

# Freeze the layers of Xception model expect batch normalization
for layer in model.layers:
    layer.trainable = False

    if layer.name.startswith('batch_normalization'):
        layer.trainable = True
    if layer.name.endswith('bn'):
        layer.trainable = True
        
model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ["accuracy"]
)

See the model

In [ ]:
model.summary()

Create callbacks

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

earlystop = EarlyStopping(monitor = 'val_loss', 
                          patience = 10,
                          verbose = 1)

learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_loss',
                                            factor = 0.1,
                                            patience = 3,
                                            verbose = 1,
                                            min_lr=0.00001)

callbacks = [earlystop, learning_rate_reduction]

Split df into train and validate data

In [ ]:
train_df, validate_df = train_test_split(df, test_size=0.20, random_state=42)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

See the amount of train_df and validate_df

In [ ]:
train_df['category'].value_counts().plot.bar()

In [ ]:
validate_df['category'].value_counts().plot.bar()

In [ ]:
total_train = train_df.shape[0]
total_validate = validate_df.shape[0]
batch_size=32 

Data Augmentation

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

Create Train and Validate Data

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    "./train_data/", 
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "./train_data/", 
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)

Fit the model

In [ ]:
epochs=3 if FAST_RUN else 50
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size,
    callbacks=callbacks,
    shuffle = True, # 違う
    verbose= 1
)

Save weights

In [ ]:
model.save_weights("xception_adam_model.h5")

Visualize Training

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))
ax1.plot(history.history['loss'], color='b', label="Training loss")
ax1.plot(history.history['val_loss'], color='r', label="validation loss")
ax1.set_xticks(np.arange(1, epochs, 1))
ax1.set_yticks(np.arange(0, 1, 0.1))

ax2.plot(history.history['accuracy'], color='b', label="Training accuracy")
ax2.plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
ax2.set_xticks(np.arange(1, epochs, 1))

legend = plt.legend(loc='best', shadow=True)
plt.tight_layout()
plt.show()

Preparing Test Data

In [ ]:
test_filenames = os.listdir("./test_data")
test_labels = []
for test_filename in test_filenames:
  test_label = test_filename.split('.')[0]
  test_labels.append(test_label)

test_df = pd.DataFrame({
    'filename' : test_filenames,
    'label' : test_labels
})
nb_samples = test_df.shape[0]

Generate Test Data

In [ ]:
test_gen = ImageDataGenerator(rescale=1./255)
test_generator = test_gen.flow_from_dataframe(
    test_df,
    "./test_data/",
    x_col='filename',
    y_col=None,
    class_mode=None,
    target_size=IMAGE_SIZE,
    batch_size=batch_size, #batch_sizeが違う
    shuffle=False
)

Predict

In [ ]:
predict = model.predict(test_generator, steps=np.ceil(nb_samples/batch_size))

test_df['category'] = np.argmax(predict, axis=-1)
test_df

label_map = dict((v, k) for k, v in train_generator.class_indices.items())
test_df['category'] = test_df['category'].replace(label_map)

Visualize result

In [ ]:
test_df['category'].value_counts().plot.bar()

Confusion matrix

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
  if normalize:
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print("Normalized confusion matrix")
  else:
    print('Confusion matrix, without normalization')

  print(cm)

  plt.imshow(cm, interpolation='nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=90)
  plt.yticks(tick_marks, classes)

  fmt = '.2f' if normalize else 'd'
  thresh = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], fmt),
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')

confusion_mtx = confusion_matrix(test_df['label'], test_df['category'])
plot_confusion_matrix(confusion_mtx, classes = range(7))

Caliculate F1 score(Accuracy)

In [ ]:
from sklearn.metrics import f1_score
point = f1_score(test_df['label'], test_df['category'], average='micro')


Convert to Percentage

In [ ]:
print('accuracyは' + str(100 * point) + '%')

Submission

In [ ]:
submission_df = test_df.copy()
submission_df['id'] = submission_df['filename'].str.split('.').str[0]
submission_df['label'] = submission_df['category']
submission_df.drop(['filename', 'category'], axis=1, inplace=True)
submission_df.to_csv('submission.csv', index=False)